# Portfolio Optimization (Sharpe Ratio) Some Ideas

We are going to use the Python Library PyPotfolioOpt. The documentation of the library can be found in the following link.  https://pypi.org/project/pyportfolioopt/#description

In [1]:
import pandas_datareader.data as web
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
start = datetime.datetime(2019,9,15)
end = datetime.datetime(2022,2,15)  # This is the start/end dates for the price of stocks

In [7]:
def get_stock(ticker):
    data = web.DataReader(f"{ticker}","yahoo",start,end)
    data[f'{ticker}'] = data["Close"]
    data = data[[f'{ticker}']] 
    print(data.head())
    return data 

In [8]:
pfizer = get_stock("PFE")
jnj = get_stock("JNJ")

                  PFE
Date                 
2019-09-16  34.943073
2019-09-17  34.629982
2019-09-18  34.516129
2019-09-19  34.639469
2019-09-20  34.810246
                   JNJ
Date                  
2019-09-16  129.539993
2019-09-17  129.669998
2019-09-18  130.410004
2019-09-19  130.110001
2019-09-20  131.649994


In [10]:
from functools import reduce

def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    print(df_merged.head())
    return df_merged


stocks = ["MRNA", "PFE", "JNJ", "GOOGL", 
          "FB", "AAPL", "COST", "WMT", "KR", "JPM", 
          "BAC", "HSBC"]
portfolio = combine_stocks(stocks)

                 MRNA
Date                 
2019-09-16  17.049999
2019-09-17  17.660000
2019-09-18  17.780001
2019-09-19  17.900000
2019-09-20  18.070000
                  PFE
Date                 
2019-09-16  34.943073
2019-09-17  34.629982
2019-09-18  34.516129
2019-09-19  34.639469
2019-09-20  34.810246
                   JNJ
Date                  
2019-09-16  129.539993
2019-09-17  129.669998
2019-09-18  130.410004
2019-09-19  130.110001
2019-09-20  131.649994
                  GOOGL
Date                   
2019-09-16  1231.630005
2019-09-17  1229.880005
2019-09-18  1232.650024
2019-09-19  1238.750000
2019-09-20  1229.839966
                    FB
Date                  
2019-09-16  186.220001
2019-09-17  188.080002
2019-09-18  188.139999
2019-09-19  190.139999
2019-09-20  189.929993
                 AAPL
Date                 
2019-09-16  54.974998
2019-09-17  55.174999
2019-09-18  55.692501
2019-09-19  55.240002
2019-09-20  54.432499
                  COST
Date                  
20

In [12]:
portfolio.to_csv("portfolio.csv", index=False) #Creation of a CSV File
portfolio = pd.read_csv("portfolio.csv") 


We are going to implement the mean variance optimization method. We are going to need the covariance matrix, as well as the returns.

In [13]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage


mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

With the EfficientFrontier module we are going to calculate the weights of the basket. We are going to maximize the sharpe ratio (ratio between return and risk). The algorithm in the library looks for the maximum Sharpe ratio, which means that the constructed portfolio has the highest return and lower risk (better performance). 

In [14]:
from pypfopt.efficient_frontier import EfficientFrontier #The output of this code are the weights of the different stocks. 

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'MRNA': 0.23293, 'PFE': 0.0, 'JNJ': 0.0, 'GOOGL': 0.09047, 'FB': 0.0, 'AAPL': 0.48131, 'COST': 0.0, 'WMT': 0.0, 'KR': 0.19528, 'JPM': 0.0, 'BAC': 0.0, 'HSBC': 0.0}


In [15]:
ef.portfolio_performance(verbose=True) #Display of portfolio performance

Expected annual return: 72.0%
Annual volatility: 30.0%
Sharpe Ratio: 2.34


(0.7201117174204256, 0.29955848718290584, 2.337145323453807)

In [16]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices 

latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000) 
                                                                               
#Example of the number of shares we must buy of each stock with a Notional of 1M USD
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'AAPL': 2785, 'MRNA': 1541, 'KR': 4207, 'GOOGL': 33}
Funds remaining: $495.48


We must buy 2785 shares of Apple, 1541 of MRNA, 4207 of KR, 33 of GOOGL. 